In [9]:
import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf
import tensorflow_datasets as tfds
import tensorflow_addons as tfa
from datetime import datetime
from fgsm import  get_adversarial_exemple 

%load_ext tensorboard
try:
    %load_ext autotime
except:
    !pip install ipython-autotime
    %load_ext autotime
    

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard
The autotime extension is already loaded. To reload it, use:
  %reload_ext autotime
time: 2.81 ms (started: 2022-12-22 21:25:53 +01:00)


In [2]:
(ds_train, ds_test), ds_info = tfds.load(
    'mnist',
    split=['train', 'test'],
    shuffle_files=True,
    as_supervised=True,
    with_info=True,
)
def normalize_img(image, label):
  """Normalizes images: `uint8` -> `float32`."""
  return tf.cast(image, tf.float32) / 255., label

ds_train = ds_train.map(
    normalize_img, num_parallel_calls=tf.data.AUTOTUNE)
ds_train = ds_train.cache()
ds_train = ds_train.shuffle(ds_info.splits['train'].num_examples)
ds_train = ds_train.batch(128)
ds_train = ds_train.prefetch(tf.data.AUTOTUNE)

ds_test = ds_test.map(
    normalize_img, num_parallel_calls=tf.data.AUTOTUNE)
ds_test = ds_test.batch(128)
ds_test = ds_test.cache()
ds_test = ds_test.prefetch(tf.data.AUTOTUNE)


2022-12-22 21:15:23.705420: E tensorflow/compiler/xla/stream_executor/cuda/cuda_driver.cc:267] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
2022-12-22 21:15:23.705488: I tensorflow/compiler/xla/stream_executor/cuda/cuda_diagnostics.cc:163] no NVIDIA GPU device is present: /dev/nvidia0 does not exist
2022-12-22 21:15:23.706145: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


time: 220 ms (started: 2022-12-22 21:15:23 +01:00)


In [5]:
# convert tf.data.Dataset to numpy array
x_test = np.concatenate([x for x, y in ds_test], axis=0)
y_test = np.concatenate([y for x, y in ds_test], axis=0)
x_train = np.concatenate([x for x, y in ds_train], axis=0)
y_train = np.concatenate([y for x, y in ds_train], axis=0)

time: 1.57 s (started: 2022-12-22 21:20:10 +01:00)


In [3]:

class SequentialAdversarialLoss(tf.keras.Sequential):
    def __init__(self, *args, **kwargs):
        super(SequentialAdversarialLoss, self).__init__(*args, **kwargs)
        self.alpha = 0.5
        self.a = []
        self.eps = 0.25
    
    def train_step(self, data):
        # Unpack the data. Its structure depends on your model and
        # on what you pass to `fit()`.
        x, y = data
        """for i in x:
            print(x)
        print(tf.identity(x))
        dataset = tf.data.Dataset.from_tensor_slices((tf.expand_dims(tf.identity(x),0),tf.expand_dims(tf.identity(y),0)))
        print(dataset)
        print("-------------------")
        adv_x = get_adversarial_exemple(self,dataset,0.25)
        print("++++++++++++++++++++++")"""

        with tf.GradientTape(persistent=True) as tape:
            tape.watch(x)
            #tape.watch(adv_x)
            y_pred = self(x, training=True)  # Forward pass
            
            #adv_y_pred = self(adv_x, training=True)  # Forward pass
            # Compute the loss value
            # (the loss function is configured in `compile()`)
            loss = self.compiled_loss(y, y_pred)
            
        gradient = tape.gradient(loss, x)
        signed_grad = tf.sign(gradient)
        x_adv = x + 0.25 * signed_grad
        x_adv = tf.clip_by_value(x_adv, 0, 1)
        
        with tf.GradientTape(persistent=True) as tape:
            tape.watch(x_adv)
            y_pred_adv = self(x_adv, training=True)  # Forward pass
            # Compute the loss value
            # (the loss function is configured in `compile()`)
            loss *= self.alpha 
            loss += (1 - self.alpha) * self.compiled_loss(y, y_pred_adv)
            
            
        # Compute gradients
        trainable_vars = self.trainable_variables
        gradients = tape.gradient(loss, trainable_vars)

        # Update weights
        self.optimizer.apply_gradients(zip(gradients, trainable_vars))
        del tape
        # Update metrics (includes the metric that tracks the loss)
        self.compiled_metrics.update_state(y, y_pred)
        # Return a dict mapping metric names to current value

        return {m.name: m.result() for m in self.metrics}
    

time: 2.18 ms (started: 2022-12-22 21:15:23 +01:00)


In [6]:
# Shallow maxout model inspired from https://github.com/philipperemy/tensorflow-maxout/blob/master/mnist_maxout_example.py

shallow_maxout = SequentialAdversarialLoss([
  tf.keras.layers.Flatten(input_shape=(28, 28)),
  tf.keras.layers.Dense(960),
  tfa.layers.Maxout(240),
  tf.keras.layers.Dropout(0.6),
  tf.keras.layers.Dense(960),
  tfa.layers.Maxout(240),
  tf.keras.layers.Dropout(0.6),
  tf.keras.layers.Dense(10, activation='softmax'),
])

shallow_maxout.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

shallow_maxout.fit(ds_train, epochs=20)

print("------------------Train dataset-------------")
shallow_maxout.evaluate(ds_train) 
x_train_adv = get_adversarial_exemple(shallow_maxout, ds_train, eps=0.25)
_, accuracy =  shallow_maxout.evaluate(x_train_adv, y_train, verbose = 0)
confidence = np.mean(np.max(shallow_maxout.predict(x_train_adv, verbose = 0), axis=1))
print(f"Accuracy on adversarial examples: {accuracy} \n with a mean confidence of {confidence}", )
print("------------------Test dataset-------------")
shallow_maxout.evaluate(ds_test) 
x_test_adv = get_adversarial_exemple(shallow_maxout, ds_test, eps=0.25)
loss, accuracy =  shallow_maxout.evaluate(x_test_adv, y_test, verbose = 0)
confidence = np.mean(np.max(shallow_maxout.predict(x_test_adv, verbose = 0), axis=1))
print(f"Accuracy on adversarial examples: {accuracy} \n with a mean confidence of {confidence}", )

Epoch 1/20
469/469 [==============================] - 13s 24ms/step - loss: 1.3279 - accuracy: 0.7286
Epoch 2/20
469/469 [==============================] - 12s 24ms/step - loss: 1.0089 - accuracy: 0.8477
Epoch 3/20
469/469 [==============================] - 11s 24ms/step - loss: 0.9041 - accuracy: 0.8744
Epoch 4/20
469/469 [==============================] - 11s 24ms/step - loss: 0.8366 - accuracy: 0.8917
Epoch 5/20
469/469 [==============================] - 11s 24ms/step - loss: 0.7928 - accuracy: 0.8974
Epoch 6/20
469/469 [==============================] - 12s 24ms/step - loss: 0.7617 - accuracy: 0.9017
Epoch 7/20
469/469 [==============================] - 11s 24ms/step - loss: 0.7292 - accuracy: 0.9098
Epoch 8/20
469/469 [==============================] - 11s 24ms/step - loss: 0.7053 - accuracy: 0.9132
Epoch 9/20
469/469 [==============================] - 11s 24ms/step - loss: 0.6848 - accuracy: 0.9154
Epoch 10/20
469/469 [==============================] - 11s 24ms/step - loss: 0.665

In [10]:
logdir = "logs/scalars/" + datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=logdir)

shallow_maxout = tf.keras.models.Sequential([
  tf.keras.layers.Flatten(input_shape=(28, 28)),
  tf.keras.layers.Dense(960),
  tfa.layers.Maxout(240),
  tf.keras.layers.Dropout(0.6),
  tf.keras.layers.Dense(960),
  tfa.layers.Maxout(240),
  tf.keras.layers.Dropout(0.6),
  tf.keras.layers.Dense(10, activation='softmax'),
])

shallow_maxout.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

shallow_maxout.fit(ds_train, epochs=10,callbacks=[tensorboard_callback])

print("------------------Train dataset-------------")
shallow_maxout.evaluate(ds_train) 
x_train_adv = get_adversarial_exemple(shallow_maxout, ds_train, eps=0.25)
_, accuracy =  shallow_maxout.evaluate(x_train_adv, y_train, verbose = 0)
print(f"Accuracy on adversarial examples: {accuracy}")
#confidence = np.mean(np.max(shallow_maxout.predict(x_train_adv, verbose = 0), axis=1))
#print(f"Accuracy on adversarial examples: {accuracy} \n with a mean confidence of {confidence}", )

print("------------------Test dataset-------------")
shallow_maxout.evaluate(ds_test) 
x_test_adv = get_adversarial_exemple(shallow_maxout, ds_test, eps=0.25)
loss, accuracy =  shallow_maxout.evaluate(x_test_adv, y_test, verbose = 0)
print(f"Accuracy on adversarial examples: {accuracy}")
#confidence = np.mean(np.max(shallow_maxout.predict(x_test_adv, verbose = 0), axis=1))
#print(f"Accuracy on adversarial examples: {accuracy} \n with a mean confidence of {confidence}", )

Epoch 1/10
469/469 [==============================] - 8s 15ms/step - loss: 0.4303 - accuracy: 0.8658
Epoch 2/10
469/469 [==============================] - 7s 15ms/step - loss: 0.1979 - accuracy: 0.9419
Epoch 3/10
469/469 [==============================] - 7s 15ms/step - loss: 0.1567 - accuracy: 0.9543
Epoch 4/10
469/469 [==============================] - 7s 15ms/step - loss: 0.1382 - accuracy: 0.9585
Epoch 5/10
469/469 [==============================] - 7s 15ms/step - loss: 0.1212 - accuracy: 0.9644
Epoch 6/10
469/469 [==============================] - 7s 15ms/step - loss: 0.1160 - accuracy: 0.9658
Epoch 7/10
469/469 [==============================] - 7s 15ms/step - loss: 0.1084 - accuracy: 0.9681
Epoch 8/10
469/469 [==============================] - 7s 15ms/step - loss: 0.1004 - accuracy: 0.9705
Epoch 9/10
469/469 [==============================] - 7s 15ms/step - loss: 0.0951 - accuracy: 0.9715
Epoch 10/10
469/469 [==============================] - 7s 15ms/step - loss: 0.0923 - accura

In [ ]:
%tensorboard --logdir='logs/scalars'